In [1]:
!pip install semantic-kernel

^C


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached protobuf-6.31.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached isodate-0.7.2-py3-none-any.whl.metadata (11 kB)
  Using cached jsonschema_specifications-2025.4.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rfc3339_validator-0.1.4-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached chardet-5.2.0-py3-none-any.whl.metadata (3.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached pyee-13.0.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached msal-1.32.3-py3-none-any.whl.metadata (11 kB)
  Using cached msal_extensions-1.3.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached PyJWT-2.10.1-py3-none-any.whl.metadata (4.0 kB)
   -------------------------------

  DEPRECATION: Building 'pybars4' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pybars4'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  DEPRECATION: Building 'PyMeta3' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'PyMeta3'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
import asyncio

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

from semantic_kernel.connectors.ai.google.google_ai import GoogleAIChatCompletion

chat_completion_service = GoogleAIChatCompletion(
    gemini_model_id="gemini-2.0-flash",
    api_key=os.getenv("GOOGLE_API_KEY"),
    # service_id="my-service-id", # Optional; for targeting specific services within Semantic Kernel
)


c:\Users\mzubair\miniconda3\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
# Initialize the kernel
kernel = Kernel()

# Add Azure OpenAI chat completion
kernel.add_service(chat_completion_service)

In [11]:
import logging

# Set the logging level for  semantic_kernel.kernel to DEBUG.
logging.basicConfig(
    format="[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logging.getLogger("kernel").setLevel(logging.DEBUG)

In [12]:
chat_completion : GoogleAIChatCompletion = kernel.get_service(type=ChatCompletionClientBase)

> Creating a Lights Plugin having some lights and states

In [13]:
from typing import Annotated
from semantic_kernel.functions import kernel_function

class LightsPlugin:
    lights = [
        {"id": 1, "name": "Table Lamp", "is_on": False},
        {"id": 2, "name": "Porch light", "is_on": False},
        {"id": 3, "name": "Chandelier", "is_on": True},
    ]

    @kernel_function(
        name="get_lights",
        description="Gets a list of lights and their current state",
    )
    def get_state(
        self,
    ) -> str:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function(
        name="change_state",
        description="Changes the state of the light",
    )
    def change_state(
        self,
        id: int,
        is_on: bool,
    ) -> str:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = is_on
                return light
        return None

In [14]:
# Add the plugin to the kernel
kernel.add_plugin(
    LightsPlugin(),
    plugin_name="Lights",
)

KernelPlugin(name='Lights', description=None, functions={'change_state': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='change_state', plugin_name='Lights', description='Changes the state of the light', parameters=[KernelParameterMetadata(name='id', description=None, default_value=None, type_='int', is_required=True, type_object=<class 'int'>, schema_data={'type': 'integer'}, include_in_function_choices=True), KernelParameterMetadata(name='is_on', description=None, default_value=None, type_='bool', is_required=True, type_object=<class 'bool'>, schema_data={'type': 'boolean'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyH

In [15]:
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

In [ ]:
# Create a history of the conversation
history = ChatHistory()

> Turn on the Porch light

In [ ]:


history.add_user_message("I want to know the state of my lights. Turn on the Porch light.")

In [29]:
# Get the response from the AI
result = (await chat_completion.get_chat_message_contents(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
    arguments=KernelArguments(),
))[0]

In [30]:
print(result)

I've turned on the Porch light.



In [31]:
LightsPlugin.lights

[{'id': 1, 'name': 'Table Lamp', 'is_on': False},
 {'id': 2, 'name': 'Porch light', 'is_on': True},
 {'id': 3, 'name': 'Chandelier', 'is_on': True}]

> Trying to add a new light as Hallway light with id 4 and is_on True but method is not implemented

In [32]:
history.add_user_message("I want you to write a function that will add a new light to the list of lights. Return me its name and its parameters. Then using that function, add a light as Hallway light with id 4 and is_on True.")

In [33]:
result = (await chat_completion.get_chat_message_contents(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
    arguments=KernelArguments(),
))[0]

In [34]:
print(result)

I am sorry, I cannot add a new light. I can only change the state of existing lights and get a list of lights and their current state.



> Creating the method and trying again

In [35]:
class LightsPlugin:
    lights = [
        {"id": 1, "name": "Table Lamp", "is_on": False},
        {"id": 2, "name": "Porch light", "is_on": False},
        {"id": 3, "name": "Chandelier", "is_on": True},
    ]

    @kernel_function(
        name="get_lights",
        description="Gets a list of lights and their current state",
    )
    def get_state(
        self,
    ) -> str:
        """Gets a list of lights and their current state."""
        return self.lights

    @kernel_function(
        name="change_state",
        description="Changes the state of the light",
    )
    def change_state(
        self,
        id: int,
        is_on: bool,
    ) -> str:
        """Changes the state of the light."""
        for light in self.lights:
            if light["id"] == id:
                light["is_on"] = is_on
                return light
        return None
    
    # add method to add a new light
    @kernel_function(
        name="add_light",
        description="Adds a new light to the list of lights",
    )
    def add_light(
        self,
        name: str,
        id: int,
        is_on: bool,
    ) -> str:
        """Adds a new light to the list of lights."""
        new_light = {"id": id, "name": name, "is_on": is_on}
        self.lights.append(new_light)
        return new_light
    

In [38]:
# Add the plugin to the kernel
kernel.add_plugin(
    LightsPlugin(),
    plugin_name="Lights",
)

KernelPlugin(name='Lights', description=None, functions={'add_light': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='add_light', plugin_name='Lights', description='Adds a new light to the list of lights', parameters=[KernelParameterMetadata(name='name', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), KernelParameterMetadata(name='id', description=None, default_value=None, type_='int', is_required=True, type_object=<class 'int'>, schema_data={'type': 'integer'}, include_in_function_choices=True), KernelParameterMetadata(name='is_on', description=None, default_value=None, type_='bool', is_required=True, type_object=<class 'bool'>, schema_data={'type': 'boolean'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, 

In [39]:
result = (await chat_completion.get_chat_message_contents(
    chat_history=history,
    settings=execution_settings,
    kernel=kernel,
    arguments=KernelArguments(),
))[0]

In [40]:
print(result)

OK. I have added a light. Its name is Hallway light, its id is 4, and it is on.



In [41]:
LightsPlugin.lights

[{'id': 1, 'name': 'Table Lamp', 'is_on': False},
 {'id': 2, 'name': 'Porch light', 'is_on': False},
 {'id': 3, 'name': 'Chandelier', 'is_on': True},
 {'id': 4, 'name': 'Hallway light', 'is_on': True}]